In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import csv

In [2]:
df=pd.read_csv("/kaggle/input/medium-articles/medium_articles.csv")
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [3]:
df=df[["title","text"]]
df.shape

(192368, 2)

In [4]:
df = df.drop_duplicates()
df.shape

(190827, 2)

In [5]:
df['text']=df['text'].str.replace("\n","<|n|>")

In [6]:
sample_blog=df["text"][1]
print(sample_blog)

Your Brain On Coronavirus<|n|><|n|>A guide to the curious and troubling impact of the pandemic and isolation<|n|><|n|>Photo by cottonbro from Pexels<|n|><|n|>The coronavirus pandemic frustrates and confounds epidemiologists and immunologists, even after months of study. It frustrates politicians and public health officials dealing with mask non-compliance. It frustrates everyone stuck at home, whether they lost their job or adapting to Zoom.<|n|><|n|>After exposure to the virus, it first enters the lungs, using host machinery to replicate. The virus itself is just a genetic sequence enclosed in a protein and lipid coat. It binds the ACE2 receptors on lung cells, with a spike protein located on its protein-lipid coat. This receptor, attached to the virus, trafficks into the lung cell. Here the virus hijacks the machinery of the cell to replicate, damaging lung tissue and spreading throughout the body.<|n|><|n|>The ACE2 receptor, expressed in many regions of the body, is vulnerable to fu

In [7]:

count = 0
def GenerateSummary(blog):
    global count
    count += 1
    print("Summarizing blog",count+1)
    try:
        sentences = sent_tokenize(blog)
        sentences_clean = [re.sub(r'[^a-z\s]', '', sentence.lower()) for sentence in sentences]
        stop_words = stopwords.words('english')

        sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
        w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)

        sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
        max_len = max([len(tokens) for tokens in sentence_tokens])
        sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant') for embedding in sentence_embeddings]

        similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
        for i, row_embedding in enumerate(sentence_embeddings):
            for j, column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph, max_iter=600)

        # Extract top sentences based on scores
        top_sentence = {sentence: scores[index] for index, sentence in enumerate(sentences)}
        sentNeeded = round(0.25 * len(sentences)) - 1
        top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])

        summary = ""
        for sent in sentences:
            if sent in top.keys():
                summary += sent

        return summary

    except Exception as e:
        print("An error occurred:", e)
        return float('nan')  # Return NaN if any error occurs


In [8]:
import math
filename = "articleSetfinal.csv"
fields = ["title","summary","content"]

with open(filename,'a') as csvfile:
    csvwriter=csv.writer(csvfile)
    csvwriter.writerow(fields)
    
    def callback(row):
        summary = GenerateSummary(row['text'])
        if(type(summary)!=str):
            return
        rows = [row['title'],summary,row['text']]
        csvwriter.writerow(rows)
    df.apply(callback,axis=1)

Summarizing blog 2
Summarizing blog 3
Summarizing blog 4
Summarizing blog 5
Summarizing blog 6
Summarizing blog 7
Summarizing blog 8
Summarizing blog 9
Summarizing blog 10
Summarizing blog 11
Summarizing blog 12
Summarizing blog 13
Summarizing blog 14
Summarizing blog 15
Summarizing blog 16
Summarizing blog 17
Summarizing blog 18
Summarizing blog 19
Summarizing blog 20
Summarizing blog 21
Summarizing blog 22
Summarizing blog 23
Summarizing blog 24


/opt/conda/lib/python3.10/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 25
Summarizing blog 26
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 27
Summarizing blog 28
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 29
Summarizing blog 30
Summarizing blog 31
Summarizing blog 32
Summarizing blog 33
Summarizing blog 34
Summarizing blog 35
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 36
Summarizing blog 37
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 38
Summarizing blog 39
Summarizing blog 40
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to conve

/opt/conda/lib/python3.10/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:496: RuntimeWarning: invalid value encountered in subtract
  err = np.absolute(x - xlast).sum()


An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 447
Summarizing blog 448
Summarizing blog 449
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 450
Summarizing blog 451
Summarizing blog 452
Summarizing blog 453
Summarizing blog 454
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 455
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 456
Summarizing blog 457
Summarizing blog 458
Summarizing blog 459
Summarizing blog 460
Summarizing blog 461
Summarizing blog 462
Summarizing blog 463
Summarizing blog 464
An error occurred: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarizing blog 465
An error oc


KeyboardInterrupt

